### Install the required packages

In [ ]:
%pip install python-dotenv
%pip install requests
%pip install azure-storage-blob
%pip install azure-storage-file-datalake

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "data-transfer/azure/azcopy", api_key, region_override)

##### Initialize the test case

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
from azure.storage.filedatalake import DataLakeServiceClient
from datetime import datetime, timedelta, timezone

source_container_name = 'reg-source'
destination_container_name = 'reg-destination'
directory_name = 'sourceData'

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
data_lake_client = DataLakeServiceClient.from_connection_string(account_connection_string)

account_name = blob_service_client.account_name

##### Delete any previous destination contents

In [ ]:
if (data_lake_client.get_file_system_client(destination_container_name).get_directory_client(directory_name).exists()):
    data_lake_client.get_file_system_client(destination_container_name).delete_directory(directory_name)

##### Generate Valid source and destination SAS URIs

In [ ]:
source_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

destination_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=destination_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True, write=True, delete=True, add=True, create=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

source_sas_url = f"https://{account_name}.blob.core.windows.net/{source_container_name}?{source_sas_token}"
destination_sas_url = f"https://{account_name}.blob.core.windows.net/{destination_container_name}?{destination_sas_token}"

helper.log(f"Source SAS URL: {source_sas_url}")
helper.log(f"Destination SAS URL: {destination_sas_url}")

##### Generate Expired source and destination SAS URIs

In [ ]:
source_sas_token_expired = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) - timedelta(hours=1)
)

destination_sas_token_expired = generate_container_sas(
    account_name=account_name,
    container_name=destination_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True, write=True, delete=True, add=True, create=True),
    expiry=datetime.now(timezone.utc) - timedelta(hours=1)
)

source_sas_url_expired = f"https://{account_name}.blob.core.windows.net/{source_container_name}?{source_sas_token_expired}"
destination_sas_url_expired = f"https://{account_name}.blob.core.windows.net/{destination_container_name}?{destination_sas_token_expired}"

helper.log(f"Expired Source SAS URL: {source_sas_url_expired}")
helper.log(f"Expired Destination SAS URL: {destination_sas_url_expired}")

#### Generate Invalid Souce and destination SAS URIs

In [ ]:
source_sas_token_invalid = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

destination_sas_token_invalid = generate_container_sas(
    account_name=account_name,
    container_name=destination_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True, write=True, delete=True, add=True, create=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

source_sas_url_invalid = f"https://blob.core.windows.net/{source_container_name}?{source_sas_token_invalid}"
destination_sas_url_invalid = f"https://blob.core.windows.net/{destination_container_name}?{destination_sas_token_invalid}"

helper.log(f"Invalid Source SAS URL: {source_sas_url_invalid}")
helper.log(f"Invalid Destination SAS URL: {destination_sas_url_invalid}")

##### Start the Data Transfer with recursive set to false

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url,
    "destinationSasUri": destination_sas_url,
    "recursive":False
})

assert response.status_code == 202

ingestionId = response.json['ingestionJobId']

#### Poll for the status of the job with recursive false

In [ ]:
import time

done = False

while(done != True):
    response = helper.get(f'/azure/azcopysync/{ingestionId}')
    done = response.json['jobStatus'] == "Completed" or response.json['jobStatus'] == "Failed"
    time.sleep(3)

assert response.status_code == 200
assert response.json['ingestionJobId'] == ingestionId
assert response.json['jobStatus'] == "Completed"
assert response.json['detailedStatus']["totalTransfers"] == 0

#### Start the Data Transfer with expired Source Sas

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url_expired,
    "destinationSasUri": destination_sas_url,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "SourceSasUri is expired"

#### Start the Data Transfer with expired Destination Sas

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url,
    "destinationSasUri": destination_sas_url_expired,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "DestinationSasUri is expired"

#### Start the Data Transfer with invalid Source Sas

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url_invalid,
    "destinationSasUri": destination_sas_url,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "SourceSasUri must be a valid Azure Blob SAS URI"

#### Start the Data Transfer with invalid Destination Sas

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url,
    "destinationSasUri": destination_sas_url_invalid,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "DestinationSasUri must be a valid Azure Blob SAS URI"

### ST_TC_WB_1202: Verify wrong permission level in Destination and Source SAS

##Generate SAS without Valid Permissions 

In [ ]:
source_sas_token_missingpermission = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=False),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

destination_sas_token_missingpermission = generate_container_sas(
    account_name=account_name,
    container_name=destination_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=False, list=False, write=False, delete=False, add=True, create=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

source_sas_url_missingpermission = f"https://{account_name}.blob.core.windows.net/{source_container_name}?{source_sas_token_missingpermission}"
destination_sas_url_missingpermission = f"https://{account_name}.blob.core.windows.net/{destination_container_name}?{destination_sas_token_missingpermission}"

helper.log(f"Source SAS URL: {source_sas_url}")
helper.log(f"Destination SAS URL: {destination_sas_url}")

##Running POST : DATA Transfer where Source SAS url permission is missing

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url_missingpermission,
    "destinationSasUri": destination_sas_url,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "SourceSasUri requires 'rl' permissions. Please add the following missing permission: 'l'."

##Running POST : DATA Transfer where Destination SAS url permission is missing

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url,
    "destinationSasUri": destination_sas_url_missingpermission,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "DestinationSasUri requires 'racwdl' permissions. Please add the following missing permission: 'rwdl'."

### Test Case 364896: ST_TC_WB_1202: Verify Data transfer with url other than  SAS

In [ ]:
#copied any URL from the Storage account > container with Filename
#not_an_sas_uri = "https://dlsusewbqaapim.blob.core.windows.net/rag-test-source"

##When added Non SAS uri in DestinationSasUri

response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url,
    "destinationSasUri": not_an_sas_uri,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "DestinationSasUri must have a SAS token"

response = helper.post("/azure/azcopysync", {
    "sourceSasUri": not_an_sas_uri,
    "destinationSasUri": destination_sas_url,
    "recursive":True
})

assert response.status_code == 400
assert response.json["message"] == "sourceSasUri must have a SAS token"